### A Hamiltonian Doodle
***

The July 27th Riddler Classic from [FiveThirtyEight](https://fivethirtyeight.com/features/the-perfect-doodle-puzzle-to-keep-you-busy-during-boring-meetings/) is as follows: 

> Start with an empty 5-by-5 grid of squares, and choose any square you want as your starting square. The rules for moving through the grid from there are strict:
> 
> You may move exactly three cells horizontally or vertically, or you may move exactly two cells diagonally.
> 1. You are not allowed to visit any cell you already visited.
> 2. You are not allowed to step outside the grid.
> 3. You win if you are able to visit all 25 cells.
> 
> Is it possible to win? If so, how? If not, what are the largest and smallest numbers of squares you can legally visit?




**The Punchline**: Yes! We can totally do it.  The animation below shows one possible solution starting from the lower left corner of the board. 

In [38]:
from IPython.display import HTML
HTML('<img src="https://raw.githubusercontent.com/chrisketelsen/riddlers/master/figs/July27_2018RiddlerClassic/July27_2018RiddlerClassic.gif"; width=500; align="left">')

**The Solution**: This Riddler is actually a twist on the classic [Knight's Tour Puzzle](https://en.wikipedia.org/wiki/Knight%27s_tour).  In the classic puzzle you're asked to find a sequence of moves of a Knight in chess that visits each spot on the chess board exactly once.  In our case, the allowable moves are a little different, but it's pretty much the same idea.     

The general strategy for such problems is to embed the game in an undirected graph, where each spot on the board is a vertex, and we place an edge between all pairs of vertices between which there is a legal move. We can then couch the entire problem in terms of graph theory.  

In graph theory, we refer to a path through the graph that visits each vertex exactly one time a [Hamiltonian Path](https://en.wikipedia.org/wiki/Hamiltonian_path). Unfortunately, determining if a graph contains a Hamiltonian Path can be fairly difficult, and finding such a path even more so.  In fact, the problem of determining a valid Hamiltonian Path is [NP-Complete](https://en.wikipedia.org/wiki/NP-completeness), which for us just means that if we want to solve the problem on a computer, the algorithm will likely have complexity that is exponential in the number of inputs.  

Luckily for us, because the graph associated with our problem is reasonably small, we can find a valid Hamiltonian Path in a reasonable amount of time in Python using a [backtracking](https://en.wikipedia.org/wiki/Backtracking) algorithm. The code for constructing the associated graph and finding Hamiltonian Paths can be found below. 

OK, so we found a solution starting from one particular position, but what if we start from a different position? In general, we'd need to run our code again from the new starting position to verify that such a winning path exists.  But in our case, we get lucky again.  Note that in the animation, the final position of the winning sequence is three squares to the right of the starting position.  This means that we can return to our starting position from our end position using a legal move.  For the graph, this means that the Hamiltonian Path that we've found is actually a **Hamiltonian Cycle**. For us this means that we can find a winning sequence by starting from the new position, following the same winning sequence shown above, and wrapping around from the end position to the start position when necessary. 



<br><br><br><br>
## Helper Functions
***

In [1]:
import numpy as np 
import matplotlib.pylab as plt
%matplotlib inline 

In [45]:
class GridGraph:
    """Class to store and analyze grid 
    """
    def __init__(self):
        """Build undirected graph representing board 
        """
        
        # define local to global map 
        self.loc2glob, self.glob2loc, ctr = dict(), dict(), 0 
        for ii in range(5):
            for jj in range(5):
                self.loc2glob[(ii,jj)], self.glob2loc[ctr], ctr = ctr, (ii,jj), ctr+1 
                
        # define adj list based on game rules
        self.adj = {ctr: [] for ctr in range(25)}
        for ii in range(5):
            for jj in range(5):
                for move in [(ii-3,jj), (ii+3,jj), (ii,jj-3), (ii,jj+3), (ii-2,jj+2), (ii+2,jj+2), (ii-2,jj-2), (ii+2,jj-2)]:
                    if self.loc2glob.get(move, None) is not None:
                        self.adj[self.loc2glob[(ii,jj)]].append(self.loc2glob[move])
    
    def hamiltonianCycle(self, start=0):
        """ Driver function for determining a Hamiltonian Cycle from a starting node 
        """
        
        # initialize cycle 
        self.cycle = [start] 
    
        # try to find valid cycle 
        if not self.findCycle(1):
            print("No Hamiltonian Cycle exists :(")
            return None 
        
        return self.cycle 
    
    def findCycle(self, pos):
        """ Recursive function that builds the Hamiltonian cycle
        """
        
        # If we have a full length path 
        if len(self.cycle) == len(self.adj): 
            # If it's also a cycle, we're done, otherwise we're not  
            return self.cycle[-1] in self.adj[self.cycle[0]]
            # Note: This implementation actually finds Hamiltonian Cycles. 
            # If you want to find Hamiltonian paths, just change the line above to 
            # return True 
            
        # Try valid adjacent vertices not yet in path 
        for nxt in list(set(self.adj[self.cycle[pos-1]]) - set(self.cycle)):
            # add vertex to potential cycle 
            self.cycle.append(nxt)
            # if we found a valid hamiltonian cycle, exit 
            if self.findCycle(pos+1):
                return True 
            # else backtrack and continue 
            self.cycle = self.cycle[:pos]

        # we failed to find a cycle 
        return False 
    
    def prettier_plot(self, cycle):
        """Plot the current state of the board 
        """
        
        mycolors = {"blue": "steelblue", "red": "#a76c6e", "green": "#6a9373"}
        
        # setup figure 
        fig, ax = plt.subplots(figsize=(8,8))
        
        # plot grid lines 
        for ii in range(5+1):
            for jj in range(5+1):
                ax.plot([0,5], [jj, jj], color="gray", lw=3)
                ax.plot([ii, ii], [0,5], color="gray", lw=3)
        
        # define coordinates 
        xvals = np.array([self.glob2loc[p][0]+.5 for p in cycle])
        yvals = np.array([self.glob2loc[p][1]+.5 for p in cycle])
        
        # plot starting point 
        ax.scatter([xvals[0]], [yvals[0]], s=300, marker="s", color=mycolors["red"])
        
        # plot all but the last cycle vertex 
        if len(cycle) > 1: 
            ax.scatter(xvals[1:-1], yvals[1:-1], s=300, marker="o", color="gray")
            
        # plot the last vectex 
        ax.scatter([xvals[-1]], [yvals[-1]], s=300, marker="o", color=mycolors["red"])
        
        # fill in squares that have been visited 
        for x, y in zip(xvals, yvals):
            plt.fill_between([int(x), int(x+1)], [int(y), int(y)], [int(y+1), int(y+1)], color="gray", alpha=0.25)
            
        # pretty up 
        plt.xticks([])
        plt.yticks([])
        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])       
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        
        return ax 
    
    def animate(self, start=0):
        """Create images for an animation of the soluiton 
        """
        
        cycle = self.hamiltonianCycle(start=start)
        assert(len(cycle)==25)
        
        for ii in range(25):
            fig, ax = plt.subplots(figsize=(8,8))
            ax = self.prettier_plot(cycle[:ii+1])
            plt.savefig("./figs/July27_2018RiddlerClassic/start{:d}_{:d}.png".format(start,ii),bbox_inches="tight")
            plt.close()
        
g = GridGraph()
%timeit cycle = g.hamiltonianCycle(start=0)

3.53 ms ± 371 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
